# here we weant to combine utterances 

In [354]:
import pandas as pd
import re
import numpy as np
import glob

In [356]:
def cleanse_dialogs(df):
    
    # limit to only ASCII characters
    df['dialog'] = df['dialog'].apply(lambda x: ''.join([" " if ord(i) < 32 or ord(i) > 126 else i for i in x]))
    
    # remove html tags 
    df = df.replace(r'<.*?>', '', regex=True)
    
    # remove expressions
    df = df.replace(r'\[[A-Z ,]*\]', '', regex=True)
    
    # remove only non-char dialogs
    df = df.replace(r'^\W+$', '', regex=True)
    
    # remove # in dialogs
    df = df.replace(r'#', '', regex=True)
    
    # remove empty lines
    df.replace("", np.nan, inplace=True)
    df = df.dropna()
    
    
    return df

In [357]:
def combine_dialogs_as_sentences(df):
    count=0
    combine_rows = []
    result_arr = []
    combine_dialogs = []
    combine_start_time = []
    combine_end_time = []
    combine_nummber_of_dialog = []
    is_ending = False

    for index, row in df.iterrows():
        number_of_dialog = row.number_of_dialog
        start_time = row.start_time
        end_time = row.end_time
        dialog = row.dialog

        if any(x in row.dialog[-1] for x in ['.', '!', '?']):
            is_ending = True
            combine_dialogs.append(dialog)
            combine_start_time.append(start_time)
            combine_end_time.append(end_time)
            combine_nummber_of_dialog.append(str(number_of_dialog))
        else:
            combine_dialogs.append(dialog)
            combine_start_time.append(start_time)
            combine_end_time.append(end_time)
            combine_nummber_of_dialog.append(str(number_of_dialog))


        if is_ending:
            if len(combine_dialogs)>0:
                dialog = ' '.join(combine_dialogs)
                start_time = combine_start_time[0]
                end_time = combine_end_time[-1]
                number_of_dialog = '_'.join(combine_nummber_of_dialog)

                combine_dialogs = []
                combine_start_time = []
                combine_end_time = []
                combine_nummber_of_dialog = []
                is_ending = False


        if len(combine_dialogs) < 1:
            result_arr.append({"number_of_dialog":number_of_dialog, "start_time": start_time, "end_time": end_time, "dialog": dialog})
            
    result_df = pd.DataFrame(result_arr)
    return result_df

In [363]:
# reduce single word utterances

def combine_one_word_dialog(df):
    count=0
    result_arr = []
    combine_dialogs = []
    combine_start_time = []
    combine_end_time = []
    combine_number_of_dialog = []
    is_two_word = False

    for index, row in df.iterrows():
        number_of_dialog = row.number_of_dialog
        start_time = row.start_time
        end_time = row.end_time
        dialog = row.dialog

        if re.match(r"^ *[A-Za-z-,'0-9&!?.]* *[!.?]* *$",dialog):    # 1 word
#         if re.match(r"^ *[A-Za-z-,'0-9&!?.]* ?[A-Za-z-'0-9]* *[!.?]* *$",dialog): # 2 word
            combine_dialogs.append(dialog)
            combine_start_time.append(start_time)
            combine_end_time.append(end_time)
            combine_number_of_dialog.append(str(number_of_dialog))
        else:
            is_two_word = False
            combine_dialogs.append(dialog)
            combine_start_time.append(start_time)
            combine_end_time.append(end_time)
            combine_number_of_dialog.append(str(number_of_dialog))


        if not is_two_word:
            if len(combine_dialogs)>0:
                dialog = ' '.join(combine_dialogs)
                start_time = combine_start_time[0]
                end_time = combine_end_time[-1]
                number_of_dialog = '_'.join(combine_number_of_dialog)

                combine_dialogs = []
                combine_start_time = []
                combine_end_time = []
                combine_number_of_dialog = []
                is_two_word = True


        if len(combine_dialogs) < 1:
            result_arr.append({"number_of_dialog":number_of_dialog, "start_time": start_time, "end_time": end_time, "dialog": dialog})
            
    result_df = pd.DataFrame(result_arr)
    return result_df

In [369]:
def main(path, destination):
    for filename in glob.glob(path+'*.csv'):
        print(filename)
        movie_df = pd.read_csv(filename)
        movie_df = cleanse_dialogs(movie_df)
        movie_df = combine_dialogs_as_sentences(movie_df)
        movie_df = combine_one_word_dialog(movie_df)
    
        movie_df.to_csv(destination + '/' + filename.split('\\')[-1].split('.')[0] + '.csv')

In [370]:
main(path="CSV_2/", destination='CSV_3_combined_dialogs')

CSV_2\American History X (1998).csv
CSV_2\BlacKkKlansman (2018)23.csv
CSV_2\Django.csv
CSV_2\Pulp.csv
CSV_2\South.csv
CSV_2\The Wolf of Wall Street 2013 BluRay.csv
